# Compare model outputs of AICORE DL Model


### Imports 

In [1]:
import plotly.express as px
import pandas as pd
import os
import glob
from pathlib import Path
import yaml

# workaround buggy autocomplete
%config Completer.use_jedi = False

### Functions 

In [50]:
def get_props_from_config(config_file):
    config = yaml.load(open(config_file), Loader=yaml.BaseLoader)
    m = config['model']
    try:
        resume = config['resume'][1]
    except:
        resume = ''
    return m['architecture'], m['encoder'], resume

In [37]:
def get_vals_from_dir(subdir):
    basename = subdir.name
    
    df_tr = pd.read_csv(subdir / 'train.csv')
    df_tr['basename'] = basename[:]
    df_tr['type'] = 'train'
    df_list_tr.append(df_tr)
    
    
    df_v = pd.read_csv(s / 'val.csv')
    df_v['basename'] = basename[:]
    df_v['type'] = 'val'
    df_list_v.append(df_v)
    
    return df_tr, df_v

### Settings 

In [38]:
#LOGDIR = Path('logs') / '0.3.1'
LOGDIR = Path(r'O:\p_aicore\initze\aicore-uc2_augmentation\logs')
#SUB_REGEX = '*'
SUB_REGEX = '*CV6*'

In [39]:
subdirs = list(LOGDIR.glob(SUB_REGEX))

### Data Loading 
- add config reader

In [119]:
df_list_tr = []
df_list_v = []
for s in subdirs:
    try:
        df_tr, df_v = get_vals_from_dir(s)
    except:
        continue
    config_file = s / 'config.yml'
    #try:
    architecture, backbone, resume = get_props_from_config(config_file)
    #except:
    #    architecture, backbone, resume = '', '', ''
    df_tr['architecture'] = architecture
    df_tr['backbone'] = backbone
    df_tr['a-b'] = architecture + '_' + backbone
    df_tr['resume'] = resume
    df_tr['P-R'] = df_tr['Precision'] - df_tr['Recall']
    
    df_v['architecture'] = architecture
    df_v['backbone'] = backbone
    df_v['a-b'] = architecture + '_' + backbone
    df_v['resume'] = resume
    df_v['P-R'] = df_v['Precision'] - df_v['Recall']
    
    df_list_tr.append(df_tr)
    df_list_v.append(df_v)

df_train = pd.concat(df_list_tr).reset_index()
df_val = pd.concat(df_list_v).reset_index()

In [120]:
idxs = df_train['resume'] != ''
df_train.loc[idxs, 'basename'] = df_train['resume']
df_train.loc[idxs, 'Epoch'] += 50

In [121]:
idxs = df_val['resume'] != ''
df_val.loc[idxs, 'basename'] = df_val['resume']
df_val.loc[idxs, 'Epoch'] += 50

### Data Analysis 

In [134]:
df_val.groupby(by='basename').quantile(1).sort_values(by='IoU', ascending=False).head(5)

,index,Epoch,Accuracy,Precision,Recall,F1,IoU,Loss,P-R
basename,,,,,,,,,
CV6__V__Horton__UNetPP_resnet50,49.0,100.0,0.992419,0.859298,0.732209,0.685576,0.521579,0.018911,0.620950
CV6__V__Horton__UNetPP_resnet101,49.0,95.0,0.992294,0.865004,0.747497,0.685120,0.521051,0.033582,0.489144
CV6__V__Horton__DeepLabV3_resnet50,49.0,53.0,0.992820,1.000000,0.737570,0.675205,0.509668,0.029535,0.999990
CV6__V__Horton__UNet_resnet50,49.0,100.0,0.991650,0.716633,0.866582,0.675116,0.509566,0.030915,0.405374
CV6__V__Horton__UNetPP_resnet18,49.0,100.0,0.991808,0.757581,0.905897,0.664812,0.497916,0.141120,0.473916


In [131]:
df_val.groupby(by='basename').quantile(.95).sort_values(by='IoU', ascending=False).head(5)

,index,Epoch,Accuracy,Precision,Recall,F1,IoU,Loss,P-R
basename,,,,,,,,,
CV6__V__Horton__UNet_resnet50,47.00,95.05,0.990652,0.663000,0.776900,0.661133,0.493801,0.019647,0.066063
CV6__V__Horton__UNetPP_resnet101,44.55,90.55,0.991158,0.721544,0.718836,0.656978,0.489184,0.016152,0.264889
CV6__V__Horton__DeepLabV3_resnet50,46.75,50.75,0.991816,0.839326,0.684677,0.643623,0.474519,0.020283,0.494250
CV6__V__Horton__UNet_resnet34,47.00,95.05,0.989905,0.626330,0.781050,0.640572,0.471209,0.020832,0.058125
CV6__V__Horton__UNetPP_resnet50,47.00,95.05,0.991285,0.780153,0.691066,0.635655,0.465906,0.014745,0.366737


In [132]:
df_val.groupby(by='basename').median().sort_values(by='IoU', ascending=False).head(5)

,index,Epoch,Accuracy,Precision,Recall,F1,IoU,Loss,P-R
basename,,,,,,,,,
CV6__V__Horton__UNet_resnet50,24.5,50.5,0.986830,0.502576,0.694097,0.578806,0.407268,0.011105,-0.182663
CV6__V__Horton__UNetPP_resnet101,23.0,48.0,0.988259,0.556085,0.616984,0.571428,0.400000,0.010204,-0.057157
CV6__V__Horton__UNetPP_resnet18,24.5,50.5,0.987573,0.526677,0.635282,0.570926,0.399510,0.011728,-0.115420
CV6__V__Horton__UNet_resnet34,24.5,50.5,0.985852,0.478729,0.711521,0.567907,0.396558,0.012206,-0.211252
CV6__V__Horton__DeepLabV3_resnet34,24.5,50.5,0.989084,0.612971,0.540442,0.559696,0.388118,0.013394,0.051749


### Plots 

In [84]:
# put data into plottable shape
id_vars = ['Epoch', 'basename', 'type', 'architecture', 'a-b', 'backbone']

df_train_melt = df_train.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
df_val_melt = df_val.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
# merge
df_melt = pd.concat([df_val_melt])
#df_melt = pd.concat([df_train_melt, df_val_melt])

### Training data 

In [142]:
#px.line(df_melt, x='Epoch' , y='value' , color='basename', line_group='type')
df_train_melt = df_train.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
p = px.line(df_train_melt, x='Epoch' , y='value' , color='basename', line_group='architecture')
p.update_layout(legend_orientation='h', )

### Validation data 

#### Performance per Epoch IoU

In [143]:
df_val_melt = df_val.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
p = px.line(df_val_melt.sort_values(by=['Epoch', 'architecture', 'backbone']), x='Epoch' , y='value' , color='basename', line_dash='architecture', line_group='backbone')
p.update_layout(legend_orientation='h', )

In [135]:
df_val_melt_PR = df_val.melt(id_vars=id_vars, value_vars=['P-R']).sort_values(by='Epoch')
p = px.line(df_val_melt_PR.sort_values(by=['Epoch', 'architecture', 'backbone']), x='Epoch' , y='value' , color='basename', line_dash='architecture', line_group='backbone')
p.update_layout(legend_orientation='h', )

#### Plot mean values of configurations 

In [144]:
group_vals = ['Epoch', 'architecture']
df_train_melt['ds'] = df_train_melt.apply(lambda x: x.architecture, axis=1)
mean_vals = df_train_melt.groupby(group_vals).mean().reset_index(drop=False)

In [145]:
px.line(mean_vals, x='Epoch', y='value', color=group_vals[1])

In [148]:
group_vals = ['Epoch', 'architecture']
df_val_melt['ds'] = df_val_melt.apply(lambda x: x.architecture, axis=1)
mean_vals = df_val_melt.groupby(group_vals).mean().reset_index(drop=False)

In [149]:
px.line(mean_vals, x='Epoch', y='value', color=group_vals[1])

## Experimental part 